<a href="https://colab.research.google.com/github/simon-raut/Horror-Story-Generator/blob/main/horror_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = '/content/creep_dataset.cvs'
df = pd.read_csv(csv_file_path)

# Combine titles and stories
text = ' '.join(df['title'] + ' ' + df['story'])


In [ ]:
import tensorflow as tf

# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
total_chars = len(tokenizer.word_index) + 1

# Convert text to sequences of integers
input_sequences = tokenizer.texts_to_sequences([text])[0]

# Create input and output sequences
seq_length = 100  # Length of sequences for training
examples_per_epoch = len(text) // seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(input_sequences)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Create training batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [ ]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_chars, 256, batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(total_chars)
])

# Define the loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)


In [ ]:
import os
# Configure checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Train the model
EPOCHS = 50

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/50
1584/6715 [======>.......................] - ETA: 11:19:45 - loss: 1.8387